# Persian OCR Pipeline

This notebook implements an Optical Character Recognition (OCR) pipeline for Persian text, using either a pre-trained model or Tesseract OCR as a fallback.

## 1. Setup Environment

First, let's install and import all required libraries. For Windows users, make sure you have Tesseract installed and the Persian language data file (`fas.traineddata`) placed in the tessdata directory.

In [ ]:
# Install all required packages
!pip install easyocr torch torchvision torchaudio opencv-python pillow pandas pytesseract

In [1]:
# Import all required libraries
import os
import cv2
import time
import numpy as np
from PIL import Image
from difflib import SequenceMatcher

# Deep learning imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# OCR imports
import easyocr
import pytesseract

# Initialize EasyOCR with Persian language support
reader = easyocr.Reader(['fa'])

# For Windows users: Uncomment and modify the line below with your Tesseract installation path
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

### Pre-trained Model Setup

For this project, you have several options for the pre-trained model:

1. Use EasyOCR's pre-trained Persian model (recommended for getting started)
2. Use a custom PyTorch model trained specifically for Persian text
3. Fine-tune an existing model on your specific data

Let's set up EasyOCR first as it provides good out-of-the-box performance for Persian text:

In [6]:
# Install easyocr
#!pip install easyocr

# Initialize EasyOCR and set up model saving
import easyocr
import os
import torch

# Initialize the reader with Persian language
reader = easyocr.Reader(['fa'])

# Save the model to our model directory
model_path = 'model/persian_ocr.pth'
os.makedirs('model', exist_ok=True)

def save_easyocr_model():
    """Save the EasyOCR detection model."""
    try:
        # Get the detection model (this is what we want to save)
        detector = reader.detector
        
        # Save just the model state dictionary
        torch.save({
            'detector_state': detector.state_dict(),
            'language': 'fa'
        }, model_path)
        
        print(f"Model saved to {model_path}")
        return detector
    except Exception as e:
        print(f"Error saving model: {str(e)}")
        return None

# Save the model if it doesn't exist
if not os.path.exists(model_path):
    print("Saving EasyOCR model...")
    model = save_easyocr_model()
    if model is not None:
        print("Model saved successfully!")
    else:
        print("Failed to save model")
else:
    print(f"Model already exists at {model_path}")
    
# Test that the model works
print("\nTesting model access:")
print(f"Detector architecture: {reader.detector}")
print(f"Recognition model: {reader.recognizer}")

# Test OCR on a sample image
test_image = os.path.join('handwritten_data', os.listdir('handwritten_data')[0])
result = reader.readtext(test_image)
print(f"\nTesting OCR on first image: {os.path.basename(test_image)}")
print("Results:")
for bbox, text, conf in result:
    print(f"Text: {text}")
    print(f"Confidence: {conf:.2f}")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Saving EasyOCR model...
Model saved to model/persian_ocr.pth
Model saved successfully!

Testing model access:
Detector architecture: CRAFT(
  (basenet): vgg16_bn(
    (slice1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0

d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Testing OCR on first image: 03015057820506540105001.jpg
Results:
Text: |-(
Confidence: 0.39
Text: ٣٠
Confidence: 0.64
Text: ٥( ٧
Confidence: 0.22
Text: الم (
Confidence: 0.32


In [8]:
# Inspect the EasyOCR components
print("Detector Model Architecture:")
print(reader.detector)

print("\nRecognizer Model Architecture:")
print(reader.recognizer)

# Count parameters for both models
detector_params = sum(p.numel() for p in reader.detector.parameters())
recognizer_params = sum(p.numel() for p in reader.recognizer.parameters())

print("\nModel Parameters:")
print(f"Detector parameters: {detector_params:,}")
print(f"Recognizer parameters: {recognizer_params:,}")
print(f"Total parameters: {detector_params + recognizer_params:,}")

# Save both models
model_path = 'model/persian_ocr.pth'
print(f"\nSaving models to {model_path}...")
torch.save({
    'detector_state': reader.detector.state_dict(),
    'recognizer_state': reader.recognizer.state_dict(),
    'language': 'fa'
}, model_path)
print("Models saved successfully!")

Detector Model Architecture:
CRAFT(
  (basenet): vgg16_bn(
    (slice1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (slice2): Sequential(
      (12): ReLU(inplace=True

In [9]:
# Test the model on a sample image
def test_model():
    # Get the first image from handwritten_data
    image_files = [f for f in os.listdir('handwritten_data') 
                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not image_files:
        print("No images found in handwritten_data directory")
        return
        
    test_image = os.path.join('handwritten_data', image_files[0])
    print(f"Testing OCR on {test_image}")
    
    # Run OCR
    result = reader.readtext(test_image)
    
    # Print results
    print("\nOCR Results:")
    for bbox, text, conf in result:
        print(f"Text: {text}")
        print(f"Confidence: {conf:.2f}\n")

test_model()

Testing OCR on handwritten_data\03015057820506540105001.jpg


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



OCR Results:
Text: |-(
Confidence: 0.39

Text: ٣٠
Confidence: 0.64

Text: ٥( ٧
Confidence: 0.22

Text: الم (
Confidence: 0.32



In [10]:
# Test OCR on multiple images
def test_multiple_images(num_images=3):
    # Get image files
    image_files = sorted([
        f for f in os.listdir('handwritten_data')
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ])[:num_images]
    
    print(f"Testing OCR on {len(image_files)} images:\n")
    
    for img_file in image_files:
        img_path = os.path.join('handwritten_data', img_file)
        print(f"Processing: {img_file}")
        
        # Run OCR
        result = reader.readtext(img_path)
        
        # Print results
        print("Detected text:")
        for bbox, text, conf in result:
            print(f"- {text} (confidence: {conf:.2f})")
        print("\n" + "-"*50 + "\n")

# Run the test
test_multiple_images(3)

Testing OCR on 3 images:

Processing: 03015057820506540105001.jpg


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Detected text:
- |-( (confidence: 0.39)
- ٣٠ (confidence: 0.64)
- ٥( ٧ (confidence: 0.22)
- الم ( (confidence: 0.32)

--------------------------------------------------

Processing: 03015057820506540105002.jpg


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Detected text:
- ج ( يامن دیب (confidence: 0.09)
-  الصهرال (confidence: 0.18)
- الف  (confidence: 0.21)
- دائون (confidence: 0.28)

--------------------------------------------------

Processing: 03015057820506540105003.jpg


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Detected text:
- دط (confidence: 0.25)
- ب ( (confidence: 0.34)

--------------------------------------------------



In [14]:
def test_batch():
    """Test OCR on a batch of images using EasyOCR directly"""
    # Get first 3 images from handwritten_data
    image_files = [
        os.path.join('handwritten_data', f)
        for f in os.listdir('handwritten_data')
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ][:3]
    
    if not image_files:
        print("No images found in handwritten_data directory")
        return
    
    print(f"Testing OCR on {len(image_files)} images...")
    results = []
    
    # Process each image using EasyOCR
    for img_path in image_files:
        img_name = os.path.basename(img_path)
        print(f"\nProcessing: {img_name}")
        
        try:
            # Run OCR using EasyOCR reader
            ocr_result = reader.readtext(img_path)
            
            # Combine all detected text
            text = ' '.join([r[1] for r in ocr_result])
            confidence = np.mean([r[2] for r in ocr_result]) if ocr_result else 0
            
            results.append((img_name, text))
            print(f"Confidence: {confidence:.2f}")
            
        except Exception as e:
            print(f"Error processing {img_name}: {str(e)}")
    
    # Print results
    print("\nOCR Results:")
    for img_name, text in results:
        print(f"\nImage: {img_name}")
        print(f"Text: {text}")

# Run the batch test
test_batch()


Testing OCR on 3 images...

Processing: 03015057820506540105001.jpg


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.39

Processing: 03015057820506540105002.jpg


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.19

Processing: 03015057820506540105003.jpg


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.30

OCR Results:

Image: 03015057820506540105001.jpg
Text: |-( ٣٠ ٥( ٧ الم (

Image: 03015057820506540105002.jpg
Text: ج ( يامن دیب  الصهرال الف  دائون

Image: 03015057820506540105003.jpg
Text: دط ب (


In [ ]:
# Install required packages
#!pip install torch torchvision torchaudio
#!pip install opencv-python pillow pandas pytesseract

# Note: For Windows users, download Tesseract from:
# https://github.com/UB-Mannheim/tesseract/wiki
# And fas.traineddata from:
# https://github.com/tesseract-ocr/tessdata/blob/main/fas.traineddata

In [15]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pytesseract

# Set Tesseract path for Windows users
# Uncomment and modify the line below with your Tesseract installation path
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

## 2. Directory Structure

Ensure your project has the following structure:
```
project/
├── handwritten_data/
│   ├── 1.png
│   ├── 2.png
├── model/
│   └── persian_ocr.pth  # optional pretrained model
├── ocr_pipeline.ipynb   # this notebook
```

In [16]:
# Check and create directories if they don't exist
for dir_name in ['handwritten_data', 'model']:
    os.makedirs(dir_name, exist_ok=True)

# Check if model exists
model_path = 'model/persian_ocr.pth'
use_tesseract = not os.path.exists(model_path)

## 3. Preprocessing

Implement image preprocessing steps:
1. Read image
2. Convert to grayscale
3. Resize to model input shape
4. Apply Gaussian blur and adaptive threshold
5. Normalize to [0, 1] tensor

In [17]:
def preprocess_image(image_path, target_size=(384, 64)):
    """Preprocess image for OCR.
    
    Args:
        image_path (str): Path to the input image
        target_size (tuple): Target size for resizing (width, height)
    
    Returns:
        torch.Tensor: Preprocessed image tensor
    """
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Resize
    resized = cv2.resize(gray, target_size)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(resized, (5, 5), 0)
    
    # Apply adaptive threshold
    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )
    
    # Normalize to [0, 1] tensor
    tensor = torch.FloatTensor(thresh) / 255.0
    tensor = tensor.unsqueeze(0)  # Add channel dimension
    
    return tensor

## 4. Model Handling

Handle both cases:
1. Using a pre-trained model if available
2. Falling back to Tesseract OCR if no model is available

In [18]:
def get_ocr_model():
    """Load OCR model or return None if using Tesseract."""
    if use_tesseract:
        return None
    
    # If using a pre-trained model, load it here
    # This is a placeholder - modify based on your actual model architecture
    model = torch.load(model_path)
    model.eval()
    if torch.cuda.is_available():
        model = model.to('cuda')
    return model

def perform_ocr(image_tensor, model=None):
    """Perform OCR on the preprocessed image.
    
    Args:
        image_tensor (torch.Tensor): Preprocessed image tensor
        model: Optional pre-trained model
    
    Returns:
        str: Recognized text
    """
    if model is None:
        # Convert tensor back to image for Tesseract
        img_array = (image_tensor.squeeze().numpy() * 255).astype(np.uint8)
        return pytesseract.image_to_string(img_array, lang='fas')
    
    # If using a pre-trained model
    with torch.no_grad():
        if torch.cuda.is_available():
            image_tensor = image_tensor.to('cuda')
        # This is a placeholder - modify based on your actual model architecture
        output = model(image_tensor.unsqueeze(0))
        # Implement your decoding logic here
        text = decode_output(output)
        return text

## 5. Prediction and Decoding

Implement text prediction and normalization:


In [27]:
import easyocr
import numpy as np

def get_ocr_model():
    """Get the EasyOCR reader instance."""
    global reader
    if 'reader' not in globals():
        print("Initializing EasyOCR reader...")
        reader = easyocr.Reader(['fa'])
    return reader

def perform_ocr(image_path, model=None):
    """Perform OCR on an image.
    
    Args:
        image_path (str): Path to the image file
        model: EasyOCR reader instance (optional)
    
    Returns:
        str: Recognized text
    """
    if model is None:
        model = get_ocr_model()
        
    try:
        # Run OCR
        results = model.readtext(image_path)
        
        # Combine all detected text
        text = ' '.join([r[1] for r in results])
        
        # Get average confidence
        confidence = np.mean([r[2] for r in results]) if results else 0
        print(f"Confidence: {confidence:.2f}")
        
        return text
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return ""

In [40]:
def normalize_persian_text(text):
    """Normalize Persian text by handling RTL and Unicode issues.
    
    Args:
        text (str): Input text
    
    Returns:
        str: Normalized text
    """
    # Strip whitespace
    text = text.strip()
    
    # Add any specific Persian text normalization here
    # For example, fixing Unicode overlaps or standardizing characters
    
    return text

def process_image_batch(image_paths, model, batch_size=4):
    """Process a batch of images.
    
    Args:
        image_paths (list): List of image paths
        model: EasyOCR reader instance
        batch_size (int): Batch size for processing
    
    Returns:
        list: List of (image_name, predicted_text) tuples
    """
    results = []
    total_images = len(image_paths)
    
    for i in range(0, total_images, batch_size):
        batch_paths = image_paths[i:i + batch_size]
        print(f"\nProcessing batch {i//batch_size + 1}/{(total_images-1)//batch_size + 1}")
        
        for img_path in batch_paths:
            img_name = os.path.basename(img_path)
            print(f"Processing {img_name}...")
            
            try:
                # Run OCR using EasyOCR reader
                ocr_result = model.readtext(img_path)
                
                # Combine all detected text and calculate confidence
                text = ' '.join([r[1] for r in ocr_result])
                confidence = np.mean([r[2] for r in ocr_result]) if ocr_result else 0
                
                results.append((img_name, text))
                print(f"Confidence: {confidence:.2f}")
                
            except Exception as e:
                print(f"Error processing {img_name}: {str(e)}")
                results.append((img_name, ""))
    
    return results

## 6. Writing Output

Write the OCR results to a text file:

In [29]:
def write_results(results, output_file='results.txt'):
    """Write OCR results to a file.
    
    Args:
        results (list): List of (image_name, text) tuples
        output_file (str): Path to output file
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        for image_name, text in results:
            f.write(f"{image_name} → {text}\n")

## 7. Optimizations

Implement the main processing pipeline with optimizations:

In [38]:
def main():
    # Get all image files
    image_files = [
        os.path.join('handwritten_data', f)
        for f in os.listdir('handwritten_data')
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ]
    
    if not image_files:
        print("No images found in 'handwritten_data' directory!")
        return
    
    # Load model (or None for Tesseract)
    model = get_ocr_model()
    
    # Process images in batches
    results = process_image_batch(
        image_files,
        model,
        batch_size=4  # Adjust based on your GPU memory
    )
    
    # Write results
    write_results(results)
    print(f"Processed {len(results)} images. Results written to 'results.txt'")

# Run the pipeline
if __name__ == '__main__':
    main()

TypeError: 'Reader' object is not callable

In [41]:
def process_all_images(batch_size=4):
    """Process all images in the handwritten_data directory"""
    # Get all image files
    image_files = sorted([
        os.path.join('handwritten_data', f)
        for f in os.listdir('handwritten_data')
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ])
    
    print(f"Found {len(image_files)} images to process")
    
    results = []
    for i in range(0, len(image_files), batch_size):
        batch = image_files[i:i + batch_size]
        print(f"\nProcessing batch {i//batch_size + 1}/{(len(image_files)-1)//batch_size + 1}")
        
        for img_path in batch:
            img_name = os.path.basename(img_path)
            print(f"\nProcessing {img_name}...")
            
            # Run OCR
            try:
                ocr_result = reader.readtext(img_path)
                # Combine all detected text
                text = ' '.join([r[1] for r in ocr_result])
                confidence = np.mean([r[2] for r in ocr_result]) if ocr_result else 0
                
                results.append({
                    'image': img_name,
                    'text': text,
                    'confidence': confidence
                })
                
                print(f"Confidence: {confidence:.2f}")
            except Exception as e:
                print(f"Error processing {img_name}: {str(e)}")
    
    # Save results
    save_results(results)
    create_ground_truth_template(results)
    
    return results

def save_results(results, output_file='ocr_results.txt'):
    """Save OCR results to a file"""
    with open(output_file, 'w', encoding='utf-8') as f:
        for r in results:
            f.write(f"{r['image']} → {r['text']}\n")
    print(f"\nResults saved to {output_file}")

def create_ground_truth_template(results, output_file='ground_truth.txt'):
    """Create a template file for ground truth data"""
    if os.path.exists(output_file):
        print(f"\n{output_file} already exists. Not overwriting.")
        return
        
    with open(output_file, 'w', encoding='utf-8') as f:
        for r in results:
            # Pre-fill with OCR result to make it easier to correct
            f.write(f"{r['image']} → {r['text']}\n")
    
    print(f"\nGround truth template created in {output_file}")
    print("Please review and correct the text in this file for evaluation")

# Process all images
print("Starting batch processing of all images...")
results = process_all_images(batch_size=4)

Starting batch processing of all images...
Found 239 images to process

Processing batch 1/60

Processing 03015057820506540105001.jpg...


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.39

Processing 03015057820506540105002.jpg...


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.19

Processing 03015057820506540105003.jpg...


...

Confidence: 0.14

Processing 03034058010523670524001.jpg...


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.14

Processing batch 60/60

Processing 03034058010524000133001.jpg...
Confidence: 0.00

Processing 03034058010524050079001.jpg...


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.24

Processing 03034058010524050854001.jpg...
Confidence: 0.00

Results saved to ocr_results.txt

ground_truth.txt already exists. Not overwriting.


In [43]:
def evaluate_results(results_file='ocr_results.txt', ground_truth_file='ground_truth.txt'):
    """Evaluate OCR results against ground truth"""
    if not os.path.exists(ground_truth_file):
        print(f"Ground truth file {ground_truth_file} not found.")
        print("Please fill in the ground truth data first.")
        return
    
    # Load results
    ocr_results = {}
    with open(results_file, 'r', encoding='utf-8') as f:
        for line in f:
            if '→' in line:
                img, text = line.strip().split(' → ')
                ocr_results[img] = text
    
    # Load ground truth
    ground_truth = {}
    with open(ground_truth_file, 'r', encoding='utf-8') as f:
        for line in f:
            if '→' in line:
                img, text = line.strip().split(' → ')
                ground_truth[img] = text
    
    # Calculate metrics
    total_chars_acc = 0
    total_words_acc = 0
    total_images = 0
    
    print("\nEvaluation Results:")
    print("-" * 50)
    
    for img in ground_truth:
        if img in ocr_results:
            gt_text = ground_truth[img]
            ocr_text = ocr_results[img]
            
            char_acc, word_acc = calculate_metrics(ocr_text, gt_text)
            total_chars_acc += char_acc
            total_words_acc += word_acc
            total_images += 1
            
            print(f"\nImage: {img}")
            print(f"OCR Text: {ocr_text}")
            print(f"Ground Truth: {gt_text}")
            print(f"Character Accuracy: {char_acc:.2%}")
            print(f"Word Accuracy: {word_acc:.2%}")
    
    if total_images > 0:
        print("\nOverall Metrics:")
        print(f"Average Character Accuracy: {total_chars_acc/total_images:.2%}")
        print(f"Average Word Accuracy: {total_words_acc/total_images:.2%}")
        print(f"Total Images Evaluated: {total_images}")
    else:
        print("No matching images found between results and ground truth")

# Note: Run this cell after filling in ground_truth.txt
print("To evaluate the results:")
print("1. Check ocr_results.txt for the OCR output")
print("2. Review and correct ground_truth.txt")
print("3. Run evaluate_results() to see the accuracy metrics")

To evaluate the results:
1. Check ocr_results.txt for the OCR output
2. Review and correct ground_truth.txt
3. Run evaluate_results() to see the accuracy metrics


## Usage Example

1. Place your Persian text images in the `handwritten_data` directory
2. If you have a pre-trained model, place it in `model/persian_ocr.pth`
3. Run all cells in this notebook
4. Check the results in `results.txt`

## 8. Evaluation Metrics

Implement metrics to evaluate OCR accuracy and performance:
1. Character Accuracy (CA)
2. Word Accuracy (WA)
3. Processing Speed
4. Model Flexibility

In [44]:
import time
from difflib import SequenceMatcher

def levenshtein_distance(s1, s2):
    """Calculate the Levenshtein distance between two strings.
    
    Args:
        s1 (str): First string
        s2 (str): Second string
    
    Returns:
        int: Edit distance between the strings
    """
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [45]:
def calculate_metrics(predicted_text, ground_truth):
    """Calculate character and word accuracy metrics.
    
    Args:
        predicted_text (str): OCR output text
        ground_truth (str): Ground truth text
    
    Returns:
        tuple: (character_accuracy, word_accuracy)
    """
    # Character accuracy
    char_distance = levenshtein_distance(predicted_text, ground_truth)
    total_chars = max(len(predicted_text), len(ground_truth))
    char_accuracy = 1 - (char_distance / total_chars) if total_chars > 0 else 0
    
    # Word accuracy
    pred_words = set(predicted_text.split())
    truth_words = set(ground_truth.split())
    correct_words = len(pred_words.intersection(truth_words))
    total_words = len(truth_words)
    word_accuracy = correct_words / total_words if total_words > 0 else 0
    
    return char_accuracy, word_accuracy

def evaluate_model(test_images, ground_truth_file, model=None, batch_size=4):
    """Evaluate model performance on test data.
    
    Args:
        test_images (list): List of test image paths
        ground_truth_file (str): Path to ground truth file
        model: Optional pre-trained model
        batch_size (int): Batch size for processing
    
    Returns:
        dict: Dictionary containing evaluation metrics
    """
    # Load ground truth data
    ground_truth = {}
    with open(ground_truth_file, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:  # Skip empty lines
                continue
                
            try:
                if '→' not in line:
                    print(f"Warning: Line {line_num} does not contain '→' separator: {line}")
                    continue
                    
                parts = line.split('→')
                if len(parts) != 2:
                    print(f"Warning: Line {line_num} is not in the correct format 'image → text': {line}")
                    continue
                    
                img_name, text = parts
                text = text.strip()
                if not text or text == '<ground_truth_text>':
                    print(f"Warning: Line {line_num} has no valid ground truth text: {line}")
                    continue
                    
                ground_truth[img_name] = text
                
            except Exception as e:
                print(f"Error processing line {line_num}: {line}")
                print(f"Error details: {str(e)}")
                continue
    
    if not ground_truth:
        raise ValueError("No valid ground truth data found in the file. Please check the format.")
    
    total_time = 0
    total_char_acc = 0
    total_word_acc = 0
    processed = 0
    
    start_time = time.time()
    results = process_image_batch(test_images, model, batch_size)
    total_time = time.time() - start_time
    
    for img_name, pred_text in results:
        if img_name in ground_truth:
            char_acc, word_acc = calculate_metrics(pred_text, ground_truth[img_name])
            total_char_acc += char_acc
            total_word_acc += word_acc
            processed += 1
    
    if processed == 0:
        raise ValueError("No images could be processed. Check that image names in ground truth match your files.")
    
    metrics = {
        'avg_char_accuracy': total_char_acc / processed if processed > 0 else 0,
        'avg_word_accuracy': total_word_acc / processed if processed > 0 else 0,
        'avg_time_per_image': total_time / len(test_images),
        'total_images': len(test_images),
        'processed_images': processed
    }
    
    return metrics

In [46]:
def run_evaluation():
    """Run evaluation on test data and print results."""
    # Get test images
    test_images = [
        os.path.join('handwritten_data', f)
        for f in os.listdir('handwritten_data')
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ]
    
    if not test_images:
        print("No test images found in 'handwritten_data' directory!")
        return
    
    # Check for ground truth file
    if not os.path.exists('ground_truth.txt'):
        print("Warning: ground_truth.txt not found. Creating template file...")
        with open('ground_truth.txt', 'w', encoding='utf-8') as f:
            for img_path in test_images:
                img_name = os.path.basename(img_path)
                f.write(f"{img_name} → <ground_truth_text>\n")
        print("Please fill in ground_truth.txt with correct transcriptions")
        return
        
    # Validate ground truth file content
    valid_entries = False
    with open('ground_truth.txt', 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line and '→' in line:
                parts = line.split('→')
                if len(parts) == 2 and parts[1] != '<ground_truth_text>':
                    valid_entries = True
                    break
                    
    if not valid_entries:
        print("Warning: No valid entries found in ground_truth.txt")
        print("Please fill in the file with correct transcriptions in the format:")
        print("image.jpg → transcribed_text")
        return
    
    # Load model
    model = get_ocr_model()
    
    try:
        # Run evaluation
        print("Running evaluation...")
        metrics = evaluate_model(test_images, 'ground_truth.txt', model)
        
        # Print results
        print("\nEvaluation Results:")
        print(f"Character Accuracy: {metrics['avg_char_accuracy']:.2%}")
        print(f"Word Accuracy: {metrics['avg_word_accuracy']:.2%}")
        print(f"Average Processing Time: {metrics['avg_time_per_image']:.3f}s per image")
        print(f"Total Images Processed: {metrics['processed_images']}/{metrics['total_images']}")
        
        # Check performance targets
        if metrics['avg_time_per_image'] > 1.0:
            print("\nWarning: Processing time exceeds target of 1s per image")
        if metrics['avg_char_accuracy'] < 0.9:
            print("\nWarning: Character accuracy below 90% threshold")
            
    except Exception as e:
        print(f"\nError during evaluation: {str(e)}")
        print("Please check that ground_truth.txt is properly formatted and contains valid transcriptions.")

if __name__ == '__main__':
    run_evaluation()

Running evaluation...

Processing batch 1/60
Processing 03015057820506540105001.jpg...


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.39
Processing 03015057820506540105002.jpg...


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.19
Processing 03015057820506540105003.jpg...


...

Confidence: 0.14
Processing 03034058010523670524001.jpg...


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.14

Processing batch 60/60
Processing 03034058010524000133001.jpg...
Confidence: 0.00
Processing 03034058010524050079001.jpg...
Confidence: 0.00
Processing 03034058010524050079001.jpg...


d:\arfa\code\ocr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confidence: 0.24
Processing 03034058010524050854001.jpg...
Confidence: 0.00

Error during evaluation: No images could be processed. Check that image names in ground truth match your files.
Please check that ground_truth.txt is properly formatted and contains valid transcriptions.
Confidence: 0.00

Error during evaluation: No images could be processed. Check that image names in ground truth match your files.
Please check that ground_truth.txt is properly formatted and contains valid transcriptions.


## Evaluation Instructions

1. Place test images in the `handwritten_data` directory
2. Create `ground_truth.txt` with correct transcriptions in the format:
   ```
   image1.png → اين متن درست است
   image2.jpg → متن ديگر
   ```
3. Run the evaluation cells above
4. Review the metrics in the output:
   - Character Accuracy (target: >90%)
   - Word Accuracy
   - Processing Speed (target: <1s/image)
   - Number of successfully processed images